In [1]:
import ray

In [2]:
ray.shutdown()

In [3]:
ray.init()

2021-01-28 15:01:35,154	INFO services.py:1166 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.1.37',
 'raylet_ip_address': '192.168.1.37',
 'redis_address': '192.168.1.37:6379',
 'object_store_address': '/tmp/ray/session_2021-01-28_15-01-34_643480_33619/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-28_15-01-34_643480_33619/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-01-28_15-01-34_643480_33619',
 'metrics_export_port': 55854}

In [4]:
%config IPCompleter.use_jedi = False

In [41]:
import numpy as np
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import Binarizer

transformer = FunctionTransformer(np.log1p)
binarizer = Binarizer(threshold=0.5)

In [42]:
import com.ibm.research.ray.graph.Datamodel as dm

pipeline = dm.Pipeline()

node_a = dm.Node('a', transformer)
node_b = dm.Node('b', transformer)
node_c = dm.Node('c', transformer)
node_d = dm.Node('d', binarizer)

pipeline.add_edge(node_a, node_b)
pipeline.add_edge(node_b, node_c)
pipeline.add_edge(node_b, node_d)

In [34]:
X = np.array([0, 1, 2, 3])

In [23]:
transformer.transform(transformer.transform(transformer.transform(X)))

array([0.        , 0.42303586, 0.55461836, 0.62580029])

In [24]:
xref = ray.put(X)

In [35]:
import com.ibm.research.ray.graph.Runtime as rt

In [36]:
in_args={node_a: [xref]}

In [43]:
out_args = rt.execute_pipeline(pipeline, in_args)

In [44]:
for key in out_args.keys():
    print(str(key))

c
d


In [45]:
ray.get(out_args[node_c])

[array([0.        , 0.42303586, 0.55461836, 0.62580029])]

In [46]:
ray.get(out_args[node_d])

RayTaskError(ValueError): [36mray::execute_node()[39m (pid=33653, ip=192.168.1.37)
  File "python/ray/_raylet.pyx", line 484, in ray._raylet.execute_task
  File "/Users/rganti/PycharmProjects/ray-graphs/com/ibm/research/ray/graph/Runtime.py", line 9, in execute_node
    return node.get_transformer().transform(args)
  File "/Users/rganti/PycharmProjects/ray-graphs/venv37/lib/python3.7/site-packages/sklearn/preprocessing/_data.py", line 2207, in transform
    reset=False)
  File "/Users/rganti/PycharmProjects/ray-graphs/venv37/lib/python3.7/site-packages/sklearn/base.py", line 421, in _validate_data
    X = check_array(X, **check_params)
  File "/Users/rganti/PycharmProjects/ray-graphs/venv37/lib/python3.7/site-packages/sklearn/utils/validation.py", line 63, in inner_f
    return f(*args, **kwargs)
  File "/Users/rganti/PycharmProjects/ray-graphs/venv37/lib/python3.7/site-packages/sklearn/utils/validation.py", line 641, in check_array
    "if it contains a single sample.".format(array)

In [48]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

In [49]:
scaler.transform(X)

NotFittedError: This MinMaxScaler instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.

In [ ]:
if not l:
    print('True')